In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, T5EncoderModel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# 1. Load QQP dataset
dataset = load_dataset("glue", "qqp")

# 4. Create DataLoaders
train_loader = DataLoader(dataset["train"], batch_size=8, shuffle=True)
val_loader = DataLoader(dataset["validation"], batch_size=8)

# 5. Define classification model using T5 encoder + linear head
class QQPPromptDataset(Dataset):
    def __init__(self, hf_dataset_split):
        self.q1 = hf_dataset_split["question1"]
        self.q2 = hf_dataset_split["question2"]
        self.labels = hf_dataset_split["label"]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Embed the questions in a prompt-like sentence
        prompt = f"Are these questions asking the same thing? Question1: {self.q1[idx]} Question2: {self.q2[idx]}"
        label = self.labels[idx]
        return prompt, label

train_dataset = QQPPromptDataset(dataset["train"])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [2]:
max_len = 0
len_arr = []
for i in train_dataset:
    prompt, label = i

    len_arr.append(len(prompt))
    if len(prompt) > max_len:
        max_len = len(prompt)

print(f"Max length of prompt: {max_len}")
print(f"Average length of prompt: {sum(len_arr) / len(len_arr)}")


Max length of prompt: 1390
Average length of prompt: 185.82213079159865


In [2]:
tokenized_dataset.save_to_disk("tokenized_qqp_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/363846 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/40430 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = load_from_disk("tokenized_qqp_dataset")
